## We are missing data about drive - let's fix this

In [1]:
import requests
import time

api_key = '8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ'
getstr_options = 'https://developers.ria.com/auto/categories/1/driverTypes?api_key=' + api_key
getstr_option_max = 'https://developers.ria.com/auto/search?api_key='\
                    + api_key \
                    + '&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0'\
                    + '&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10' \
                    + '&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page=1&drive_type='

resp = requests.get(getstr_options)
options = resp.json()
resp.close()
folder = 'drive/'



## Get info about how many pages of ids we should retreive
### run once!

In [2]:
for pair in options:
    name = pair['name']
    value = pair['value']
    print(name + ' - ' + str(value))
    name_file = open(folder + name + '.txt', 'w', encoding='utf-8')
    name_last_file = open(folder + name + '_last_page.txt', 'w', encoding='utf-8')
    name_max_page_file = open(folder + name + '_max_page.txt', 'w', encoding='utf-8')
    resp = requests.get(getstr_option_max+str(value))
    max_page = resp.json()['result']['search_result']['count'] // 100
    print(max_page)
    resp.close()
    name_max_page_file.write(str(max_page))

    name_max_page_file.close()
    name_file.close()
    name_last_file.close()

Полный - 1
256
Передний - 2
1089
Задний - 3
281


In [3]:
str_get_id = 'https://developers.ria.com/auto/search?api_key=' + api_key \
+ '&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0'\
+'&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10'\
+ '&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3' + \
'&custom=0&page='

str_get_id

'https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page='

## Get ids of cars with certain drive type

In [4]:
for pair in options:
    name = pair['name']
    value = pair['value']
    
    name_max_page_file = open(folder + name + '_max_page.txt', 'r', encoding='utf-8')
    max_page_str = name_max_page_file.read()
    max_page = int(max_page_str.splitlines()[0])
    print('For ' + name + ' max_page is - ' + str(max_page))
    name_max_page_file.close()
    
    name_file = open(folder + name + '.txt', 'r', encoding='utf-8')
    ids_str = name_file.read()
    already_retreived = set(ids_str.splitlines())
    name_file.close()
    
    name_last_file = open(folder + name + '_last_page.txt', 'r', encoding='utf-8')
    ids_str = name_last_file.read()
    ids_str = ids_str.splitlines()
    page = 0
    if len(ids_str) != 0:
        page = int(ids_str[-1])
    name_last_file.close()
    print('For ' + name + ' start_page is - ' + str(page))
    
    retrieved_ids_file = open(folder + name + '.txt', 'a', encoding='utf-8')
    last_page_file = open(folder + name + '_last_page.txt', 'a', encoding='utf-8')
    
    try:
        while page <= 10:# use max_page if not demo
            getstr = str_get_id + str(page) + '&drive_type=' + str(value)
            print(getstr)
            resp = requests.get(getstr)

            print('request №' + str(page) + ' was sent')
            print(resp.status_code)

            if resp.status_code != 200:
                resp.close()
                continue
            else:
                ids = resp.json()['result']['search_result']['ids']
                for ido in ids:
                    if ido not in already_retreived:
                        retrieved_ids_file.write(ido)
                        retrieved_ids_file.write('\n')
                        already_retreived.add(ido)
                resp.close()
            page += 1
            time.sleep(0.2)
    finally:
        last_page_file.write(str(page))
        last_page_file.write('\n')
        last_page_file.flush()
        last_page_file.close()
        retrieved_ids_file.flush()
        retrieved_ids_file.close()

For Полный max_page is - 256
For Полный start_page is - 0
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page=0&drive_type=1
request №0 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page=1&drive_type=1
request №1 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=10000

https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page=1&drive_type=3
request №1 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=0&page=2&drive_type=3
request №2 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]